In [416]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [417]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from scipy.stats import wasserstein_distance, pearsonr, spearmanr

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import string
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [418]:
from torchtext import data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [419]:
# from torchtext.vocab import FastText
# vectors = FastText('simple')

In [420]:
BATCH_SIZE = 128
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [458]:
train_path = '/content/drive/MyDrive/emotion detection/train/'
test_path = '/content/drive/MyDrive/emotion detection/test/'
train_csv = 'train_df.csv'
test_csv = 'test_df.csv'

In [459]:
cls = ['text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
train_df = pd.read_csv(train_path + train_csv, index_col='index')
train_df.columns = cls
test_df = pd.read_csv(test_path + test_csv, index_col='index')
test_df.columns = cls

In [460]:
train_df.head()

,text,Anger,Disgust,Fear,Joy,Sadness,Surprise
index,,,,,,,
1,Mortar assault leaves at least 18 dead,22,2,60,0,64,0
2,Goal delight for Sheva,0,0,0,93,0,38
3,Nigeria hostage feared dead is freed,18,0,52,66,20,65
4,Bombers kill shoppers,66,39,94,0,86,0
5,"Vegetables, not fruit, slow brain decline",0,0,25,26,2,46


In [461]:
train_df = train_df.drop('Disgust', axis=1)
test_df = test_df.drop('Disgust', axis=1)

In [462]:
def softmax(x):
    y = np.exp(x - np.max(x))
    f_x = y / np.sum(np.exp(x - np.max(x)))
    return f_x

In [463]:
target_cols = train_df.columns[1:]
train_df_n = train_df.copy()
train_df_n[target_cols] = train_df_n[target_cols].apply(lambda it: it/it.sum() if it.sum() else 0, axis=1)
# train_df_n[target_cols] = train_df_n[target_cols].apply(softmax, axis=1)
train_df_n['target'] = train_df_n[target_cols].apply(lambda it: list(it), axis=1)
train_df_n = train_df_n[train_df_n[target_cols].sum(axis=1) > 0]
test_df_n = test_df.copy()
test_df_n[target_cols] = test_df_n[target_cols].apply(lambda it: it/it.sum() if it.sum() else 0, axis=1)
# test_df_n[target_cols] = test_df_n[target_cols].apply(softmax, axis=1)
test_df_n['target'] = test_df_n[target_cols].apply(lambda it: list(it), axis=1)
test_df_n = test_df_n[test_df_n[target_cols].sum(axis=1) > 0]
train_df_n.head()

,text,Anger,Fear,Joy,Sadness,Surprise,target
index,,,,,,,
1,Mortar assault leaves at least 18 dead,0.150685,0.410959,0.000000,0.438356,0.000000,"[0.1506849315068493, 0.410958904109589, 0.0, 0..."
2,Goal delight for Sheva,0.000000,0.000000,0.709924,0.000000,0.290076,"[0.0, 0.0, 0.7099236641221374, 0.0, 0.29007633..."
3,Nigeria hostage feared dead is freed,0.081448,0.235294,0.298643,0.090498,0.294118,"[0.08144796380090498, 0.23529411764705882, 0.2..."
4,Bombers kill shoppers,0.268293,0.382114,0.000000,0.349593,0.000000,"[0.2682926829268293, 0.3821138211382114, 0.0, ..."
5,"Vegetables, not fruit, slow brain decline",0.000000,0.252525,0.262626,0.020202,0.464646,"[0.0, 0.25252525252525254, 0.26262626262626265..."


In [464]:
train_df_n.shape, test_df_n.shape

((246, 7), (1000, 7))

In [465]:
train_input = []
train_target = []
test_input = []
test_target = []
# preprocess
for i in train_df_n.index:
  t = train_df_n.loc[i, 'text'].lower()
  t = "".join([char for char in t if char not in string.punctuation])
  train_input.append(word_tokenize(t))
  train_target.append(list(train_df_n.loc[i, target_cols]))

for i in test_df_n.index:
  t = test_df_n.loc[i, 'text'].lower()
  t = "".join([char for char in t if char not in string.punctuation])
  test_input.append(word_tokenize(t))
  test_target.append(list(test_df_n.loc[i, target_cols]))

In [466]:
max_features = 100000

tk = Tokenizer(lower = True, filters='', num_words=max_features)
full_text = train_input + test_input
# full_text = list(test_df_n['text'].values) + list(test_df_n['text'].values)
tk.fit_on_texts(full_text)

In [467]:
word2idx = tk.get_config()['word_index']

In [468]:
word2idx = eval(word2idx)
max(word2idx.values())

3445

In [469]:
def load_glove(path):
    """
    creates a dictionary mapping words to vectors from a file in glove format.
    """
    with open(path) as f:
        glove = {}
        for line in f.readlines():
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            glove[word] = vector
        return glove
def load_glove_embeddings(path, word2idx, embedding_dim=100):
    with open(path) as f:
        embeddings = np.zeros((max(word2idx.values()) + 1, embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = word2idx.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

In [470]:
train_df_n['text'].head()

index
1       Mortar assault leaves at least 18 dead
2                       Goal delight for Sheva
3         Nigeria hostage feared dead is freed
4                        Bombers kill shoppers
5    Vegetables, not fruit, slow brain decline
Name: text, dtype: object

In [471]:
from nltk import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
custom_sw = set(['report', 'new-review', 'survey', '(updated)', 'midday-wra'])
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
for i in train_df_n.index:
  train_df_n.loc[i, 'text'] = ' '.join([lemma.lemmatize(w.lower()) for w in train_df_n.loc[i, 'text'].split(' ') if not w.lower() in stop_words | custom_sw])
for i in test_df_n.index:
  test_df_n.loc[i, 'text'] = ' '.join([lemma.lemmatize(w.lower()) for w in test_df_n.loc[i, 'text'].split(' ') if not w.lower() in stop_words | custom_sw])
train_df_n.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,Anger,Fear,Joy,Sadness,Surprise,target
index,,,,,,,
1,mortar assault leaf least 18 dead,0.150685,0.410959,0.000000,0.438356,0.000000,"[0.1506849315068493, 0.410958904109589, 0.0, 0..."
2,goal delight sheva,0.000000,0.000000,0.709924,0.000000,0.290076,"[0.0, 0.0, 0.7099236641221374, 0.0, 0.29007633..."
3,nigeria hostage feared dead freed,0.081448,0.235294,0.298643,0.090498,0.294118,"[0.08144796380090498, 0.23529411764705882, 0.2..."
4,bomber kill shopper,0.268293,0.382114,0.000000,0.349593,0.000000,"[0.2682926829268293, 0.3821138211382114, 0.0, ..."
5,"vegetables, fruit, slow brain decline",0.000000,0.252525,0.262626,0.020202,0.464646,"[0.0, 0.25252525252525254, 0.26262626262626265..."


In [472]:
max([len(i.split(' ')) for i in train_df_n['text']]), max([len(i.split(' ')) for i in test_df_n['text']])

(9, 10)

In [475]:
tmp = pd.concat((train_df_n, test_df_n))
tmp, valid_df_n = train_test_split(tmp, test_size=0.2, random_state=42, stratify = tmp['target'].apply(np.argmax))
train_df_n, test_df_n = train_test_split(tmp, test_size=2/8, random_state=42, stratify = tmp['target'].apply(np.argmax))

                                        text     Anger      Fear       Joy  \
index                                                                        
1          mortar assault leaf least 18 dead  0.150685  0.410959  0.000000   
2                         goal delight sheva  0.000000  0.000000  0.709924   
3          nigeria hostage feared dead freed  0.081448  0.235294  0.298643   
4                        bomber kill shopper  0.268293  0.382114  0.000000   
5      vegetables, fruit, slow brain decline  0.000000  0.252525  0.262626   
...                                      ...       ...       ...       ...   
1495                    nasdaq fails bid lse  0.000000  0.053571  0.000000   
1496            ex-pastor get death sentence  0.037313  0.089552  0.000000   
1497        baby born turnpike dad miss exit  0.000000  0.095652  0.295652   
1498              study link chimp 'hammers'  0.000000  0.000000  0.245614   
1499      un-google earth map climate change  0.088496  0.123894

In [476]:
train_tokenized = tk.texts_to_sequences(train_df_n['text'].fillna('missing'))
test_tokenized = tk.texts_to_sequences(test_df_n['text'].fillna('missing'))
valid_tokenized = tk.texts_to_sequences(valid_df_n['text'].fillna('missing'))

max_len = 72
maxlen = 72
X_train = torch.tensor(pad_sequences(train_tokenized, maxlen = max_len), dtype=torch.long)
X_test = torch.tensor(pad_sequences(test_tokenized, maxlen = max_len), dtype=torch.long)
X_valid = torch.tensor(pad_sequences(valid_tokenized, maxlen = max_len), dtype=torch.long)

In [477]:
y_train = torch.tensor(train_df_n[target_cols].values, dtype=torch.float32)
y_test = torch.tensor(test_df_n[target_cols].values, dtype=torch.float32)
y_valid = torch.tensor(valid_df_n[target_cols].values, dtype=torch.float32)

In [478]:
y_test.shape, y_valid.shape

(torch.Size([249, 5]), torch.Size([250, 5]))

In [479]:
train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)
valid = torch.utils.data.TensorDataset(X_valid, y_valid)
    
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=y_test.shape[0], shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=y_valid.shape[0], shuffle=False)

In [480]:
# Bi-LSTM(Attention) Parameters
embedding_dim = 100
n_hidden = 100
num_classes = len(train_target[0])
words = set([c for sublist in train_input+test_input for c in sublist])
vocab_size = len(words)

In [481]:
glove_twitter_path = '/content/drive/MyDrive/emotion detection/glove.twitter.27B.100d.txt'
glove_wiki_path = '/content/drive/MyDrive/emotion detection/glove.6B.100d.txt'
# glove = load_glove(glove_wiki_path)

In [482]:
toy_embeddings = load_glove_embeddings(glove_wiki_path, word2idx)
toy_embeddings.requires_grad = False

In [483]:
# from https://github.com/sooftware/attentions/blob/master/attentions.py
class AdditiveAttention(nn.Module):
    def __init__(self, hidden_dim: int) -> None:
        super(AdditiveAttention, self).__init__()
        self.query_proj = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.key_proj = nn.Linear(hidden_dim, hidden_dim, bias=False)
        # self.bias = nn.Parameter(torch.rand(hidden_dim).uniform_(-0.1, 0.1))
        self.score_proj = nn.Linear(hidden_dim, 1)

    def forward(self, query, key, value):
        score = self.score_proj(torch.tanh(self.key_proj(key) + self.query_proj(query))).squeeze(-1)
        attn = F.softmax(score, dim=-1)
        context = torch.bmm(attn.unsqueeze(1), value)
        return context, attn

# from https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/LSTM_Attn.py

class AttentionModel(torch.nn.Module):
	def __init__(self, weights = None):
		super(AttentionModel, self).__init__()
		self.batch_size = BATCH_SIZE
		self.output_size = num_classes
		self.hidden_size = n_hidden
		self.vocab_size = vocab_size
		self.embedding_length = embedding_dim
		self.word_embeddings = nn.Embedding(vocab_size, self.embedding_length).from_pretrained(weights)
		# self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
		self.lstm = nn.LSTM(self.embedding_length, self.hidden_size, bidirectional=True)
		self.attention = AdditiveAttention(self.hidden_size * 2)
		self.label = nn.Linear(self.hidden_size * 2, self.output_size)
		self.dropout1 = nn.Dropout(0)
		self.dropout2 = nn.Dropout(0.5)
	
	def forward(self, input_sentences, batch_size=None):
		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			batch_size = self.batch_size
			h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size))
			c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size))
		else:
			h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size))
			c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size))
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		# output = output.permute(1, 0, 2)
		# attn_output = self.attention_net(self.dropout1(output), final_hidden_state)
		out = self.dropout1(final_hidden_state).permute(1, 2, 0).reshape(-1, 1, self.hidden_size * 2)
		k = torch.tile(out[:, :, -2:], (1, 1, self.hidden_size))
		attn_output = self.attention(out, k, out)[0]
		logits = self.label(self.dropout2(attn_output))
		return F.softmax(logits.squeeze(), dim = -1)
		# return torch.nn.functional.softmax(logits)

In [484]:
# from torchtext.vocab import FastText
# ft_embedding = FastText('simple')
# weights = torch.FloatTensor(ft_embedding.vectors)

# from torchtext.vocab import GloVe
# gl_embedding = GloVe('840B', 100)
# weights = torch.FloatTensor(gl_embedding.vectors)

In [485]:
EPOCHS = 100
LR = 0.0005

In [526]:
model = AttentionModel(toy_embeddings)
# model = AttentionModel(nn.Parameter(weights, requires_grad=False))
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=0.001)
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()

In [527]:
params = '/content/drive/MyDrive/emotion detection/model.pt'

In [528]:
best_param = 0
best_loss = 10000
best_metric_param = 0
best_metric = 10000
for e in range(EPOCHS):
    for i, (texts, labels) in tqdm(enumerate(train_loader)):
      optimizer.zero_grad()
      pred = model(texts, batch_size=texts.shape[0])
      loss = criterion(pred, labels)
      loss.backward()
      optimizer.step()
    with torch.no_grad():
        for i, (texts, labels) in tqdm(enumerate(valid_loader)):
          pred = model(texts, batch_size=texts.shape[0])
          loss = criterion(pred, labels)
          RMSED = np.mean([np.sqrt(nn.functional.mse_loss(pred[i:i+1, :], labels[i:i+1, :])) for i in range(pred.shape[0])])
          if loss.item() < best_loss:
              torch.save(model.state_dict(), params)
              best_loss = loss.item()
          if RMSED < best_metric:
              torch.save(model.state_dict(), params)
              best_metric = RMSED
          print(f'epoch {e}: loss: {loss.item()}, RMSED: {RMSED}')

6it [00:01,  3.71it/s]
1it [00:00,  3.89it/s]


epoch 0: loss: 0.05273080989718437, RMSED: 0.21745246648788452


6it [00:01,  3.68it/s]
1it [00:00,  3.96it/s]


epoch 1: loss: 0.05179504305124283, RMSED: 0.21544036269187927


6it [00:01,  3.73it/s]
1it [00:00,  3.56it/s]


...

epoch 97: loss: 0.03668487071990967, RMSED: 0.17337821424007416


6it [00:01,  3.54it/s]
1it [00:00,  4.43it/s]


epoch 98: loss: 0.03648851811885834, RMSED: 0.17289726436138153


6it [00:01,  3.54it/s]
1it [00:00,  4.36it/s]

epoch 99: loss: 0.0364103801548481, RMSED: 0.17235779762268066


In [529]:
best_loss, best_metric

(0.03615591302514076, 0.17162924)

In [530]:
model.load_state_dict(torch.load(params))
model.eval()

AttentionModel(
  (word_embeddings): Embedding(3446, 100)
  (lstm): LSTM(100, 100, bidirectional=True)
  (attention): AdditiveAttention(
    (query_proj): Linear(in_features=200, out_features=200, bias=False)
    (key_proj): Linear(in_features=200, out_features=200, bias=False)
    (score_proj): Linear(in_features=200, out_features=1, bias=True)
  )
  (label): Linear(in_features=200, out_features=5, bias=True)
  (dropout1): Dropout(p=0, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [531]:
def acc(x, y):
  return np.argmax(x) == np.argmax(y)

In [538]:
with torch.no_grad():
  for _, (texts, labels) in tqdm(enumerate(valid_loader)):
    pred = model(texts, batch_size=texts.shape[0])
    loss = criterion(pred, labels)
    acc_1 =  np.mean([acc(pred[i], labels[i]) for i in range(pred.shape[0])])
    APd = np.mean([pearsonr(pred[i], labels[i])[0] for i in range(pred.shape[0])])
    ASd = np.mean([spearmanr(pred[i], labels[i]) for i in range(pred.shape[0])])
    APe = np.mean([pearsonr(pred[:, i], labels[:, i])[0] for i in range(num_classes)])
    RMSED = np.mean([np.sqrt(nn.functional.mse_loss(pred[i:i+1, :], labels[i:i+1, :])) for i in range(pred.shape[0])])
    WD = np.mean([wasserstein_distance(pred[i], labels[i]) for i in range(pred.shape[0])])

1it [00:00,  1.09it/s]


In [539]:
print("---valid---\nAcc@1: ", acc_1, "\nAPd: ", APd, "\nAPe: ",APe, "\nRMSED: ",RMSED, "\nWDD: ",WD)
'------'

---valid---
Acc@1:  0.464 
APd:  0.5157066949042614 
APe:  0.5129079757088443 
RMSED:  0.17220636 
WDD:  0.10695153208673


'------'

In [540]:
with torch.no_grad():
  for _, (texts, labels) in tqdm(enumerate(test_loader)):
    pred = model(texts, batch_size=texts.shape[0])
    loss = criterion(pred, labels)
    acc_1 =  np.mean([acc(pred[i], labels[i]) for i in range(pred.shape[0])])
    APd = np.mean([pearsonr(pred[i], labels[i])[0] for i in range(pred.shape[0])])
    ASd = np.mean([spearmanr(pred[i], labels[i])[0] for i in range(pred.shape[0])])
    APe = np.mean([pearsonr(pred[:, i], labels[:, i])[0] for i in range(num_classes)])
    RMSED = np.mean([np.sqrt(nn.functional.mse_loss(pred[i:i+1, :], labels[i:i+1, :])) for i in range(pred.shape[0])])
    WD = np.mean([wasserstein_distance(pred[i], labels[i]) for i in range(pred.shape[0])])

1it [00:00,  1.19it/s]


In [541]:
loss.item(), acc_1, APd, ASd, APe, RMSED, WD

(0.03544163703918457,
 0.4819277108433735,
 0.4794549896940006,
 0.48415530209663804,
 0.4655053004005262,
 0.17285797,
 0.10580229646634744)

In [542]:
print("---test---\nAcc@1: ", acc_1, "\nAPd: ", APd, "\nAPe: ",APe, "\nRMSED: ",RMSED, "\nWDD: ",WD)
print('------')

---test---
Acc@1:  0.4819277108433735 
APd:  0.4794549896940006 
APe:  0.4655053004005262 
RMSED:  0.17285797 
WDD:  0.10580229646634744
------
